<a href="https://colab.research.google.com/github/Ayush1906saxena/DeepQLearningProjects/blob/main/Lunar_Landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 0s (7,891 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 123621 files and directories currently installed.)
Preparing to unpack .../swi

### Importing the libraries

In [3]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [4]:
class Network(nn.Module):

  def __init__(self, state_size, action_size, seed=42):
      super(Network, self).__init__()
      self.seed = torch.manual_seed(seed)
      # 64 was found to be the optimal number of neurons to land on the moon
      self.fc1 = nn.Linear(state_size, 64)
      self.fc2 = nn.Linear(64, 64)
      self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    # propagating data from input to first fully connected layer
    x = self.fc1(state)
    x = F.relu(x)

    # propagating data from first fully connected layer to second fully connected layer
    x = self.fc2(x)
    x = F.relu(x)

    #propagating data from second fully connected layer to output
    return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [5]:
import gymnasium as gym
env = gym.make('LunarLander-v3') # The Lunar Lander environment was upgraded to v3
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [6]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

### Implementing Experience Replay

In [7]:
class ReplayMemory(object):
  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    # delete the oldest event incase the memory is full
    if len(self.memory) > self.capacity:
      del self.memory[0]

  # sample method to randomly select a batch of experiences from the memory buffer
  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    # now we need to convert the states to pytorch tensors
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    # .astype(np.uint8) using this for converting to boolean values before converting it to tensors
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)

    return states, next_states, actions, rewards, dones

### Implementing the DQN class

In [10]:
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size

    # creating the local(online) and target Q networks now
    # By maintaining a separate target network, you decouple the action selection from the target Q-value generation
    self.local_qnetwork = Network(state_size, action_size).to(self.device)
    self.target_qnetwork = Network(state_size, action_size).to(self.device)

    # optimizer helps us minimize the loss function
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)

    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

  # This method will store experiences and decide when to learn from them
  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    # we want to reset every 4 step and learn from it
    self.t_step = (self.t_step + 1) % 4

    if self.t_step == 0:
      # now we learn from a minibatch of experiences
      # self.memory is our object but then memory is a variable from replay class
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  # we gonna use the epsilon-greedy this time
  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()

    # making sure no gradient computation is under progress
    with torch.no_grad():
      action_values = self.local_qnetwork(state)

    # now we are going training mode
    self.local_qnetwork.train()
    # now we do our epsilon-greedy action selection; if random number is greater than epsilon, then select max else select at random
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  # now we finally going to learn
  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    # we do (1) because we need along 1 dimension and then we get indices, value to we [0]
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_target = rewards + (discount_factor * next_q_targets * (1 - dones))
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_target)

    # now we gotta use optimizer
    self.optimizer.zero_grad()
    # backpropagate now
    loss.backward()

    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  # function to update target network's parameters
  def soft_update(self, local_model, target_model, interpolation_parameter):

    # now we gotta loop through
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

### Initializing the DQN agent

In [11]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [14]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_starting_value

# using a double ended queue
scores_on_100_episodes = deque(maxlen = 100)

# time to train the AI
for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0

  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break

  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)

  # creating an overwrite effect for printing
  # \r is used for carriage return and that end="" insures no new line gets added
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end="")

  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))

  # if score > 200, then we win
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\rEnvironment solved in {:d} episodes!\tAverageScore: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break


Episode 100	Average Score: -114.60
Episode 200	Average Score: -73.87
Episode 300	Average Score: -13.16
Episode 400	Average Score: 66.18
Episode 500	Average Score: 154.64
Episode 600	Average Score: 184.02
Environment solved in 529 episodes!	AverageScore: 200.19


## Part 3 - Visualizing the results

In [15]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()